In [1]:
import joblib
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import warnings
warnings.filterwarnings("ignore")

In [20]:
df = pd.read_csv("dados_2000.csv")

In [21]:
df

,Country,Year,Status,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,...,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
0,Afghanistan,2000,Developing,54.8,321.0,88,0.01,10.424960,62.0,6532,...,24.0,8.20,24.0,0.1,114.560000,293756.0,2.3,2.5,0.338,5.5
1,Albania,2000,Developing,72.6,11.0,1,3.66,91.711541,96.0,662,...,97.0,6.26,97.0,0.1,1175.788981,38927.0,2.1,2.2,0.656,10.7
2,Algeria,2000,Developing,71.3,145.0,21,0.25,154.455944,NaN,0,...,86.0,3.49,86.0,0.1,1757.177970,3118366.0,6.5,6.4,0.636,10.7
3,Angola,2000,Developing,45.3,48.0,97,1.85,15.881493,NaN,2219,...,3.0,2.79,28.0,2.0,555.296942,1644924.0,1.9,1.9,0.382,4.6
4,Antigua and Barbuda,2000,Developing,73.6,156.0,0,7.27,1127.743470,NaN,0,...,96.0,4.13,95.0,0.1,9875.161736,NaN,3.7,3.6,0.000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,Venezuela (Bolivarian Republic of),2000,Developing,72.5,168.0,11,8.01,0.000000,5.0,22,...,86.0,4.91,77.0,0.1,NaN,NaN,1.8,1.7,0.670,10.6
179,Viet Nam,2000,Developing,73.4,139.0,33,1.60,0.000000,NaN,16512,...,96.0,4.89,96.0,0.1,NaN,NaN,15.8,16.4,0.569,10.4
180,Yemen,2000,Developing,68.0,252.0,48,0.07,0.000000,14.0,0,...,74.0,4.14,74.0,0.1,NaN,NaN,14.1,14.1,0.436,7.7
181,Zambia,2000,Developing,43.8,614.0,44,2.62,45.616880,NaN,30930,...,85.0,7.16,85.0,18.7,341.955625,1531221.0,7.5,7.5,0.418,9.6


In [22]:
novo_df = pd.DataFrame({'country': df['Country'],
                       'life_expectancy': df['Life expectancy '],
                       'year': df['Year'],
                       'status': df['Status'],
                       'adult_mortality': df['Adult Mortality'],
                       'inf_death': df['infant deaths'],
                       'alcohol': df['Alcohol'],
                       'hepatitisB': df['Hepatitis B'],
                       'measles': df['Measles '],
                       'bmi': df[' BMI '],
                       'polio': df['Polio'],
                       'diphtheria': df['Diphtheria '],
                       'hiv': df[' HIV/AIDS'],
                       'gdp': df['GDP'],
                       'total_expenditure': df['Total expenditure'],
                       'thinness_till19': df[' thinness  1-19 years'],
                       'thinness_till9': df[' thinness 5-9 years'],
                       'school': df['Schooling'],
                       'population': df[' Population']})

In [23]:
novo_df.dtypes

country               object
life_expectancy      float64
year                   int64
status                object
adult_mortality      float64
inf_death              int64
alcohol              float64
hepatitisB           float64
measles                int64
bmi                  float64
polio                float64
diphtheria           float64
hiv                  float64
gdp                  float64
total_expenditure    float64
thinness_till19      float64
thinness_till9       float64
school               float64
population           float64
dtype: object

In [24]:
# Função para o plot da relação da variável alvo com alguns atributos
# essa funcao foi criada para que possamos visualizar algumas regressoes 
# lineares simples, da variavel alvo, com outras variáveis.
def dsa_get_pairs(data, alvo, atributos, n):
    
    # Grupos de linhas com 3 (n) gráficos por linha
    grupos_linhas = [atributos[i:i+n] for i in range(0, len(atributos), n)]

    # Loop pelos grupos de linhas para criar cada pair plot
    for linha in grupos_linhas:
        plot = sns.pairplot(x_vars = linha, y_vars = alvo, data = data, kind = "reg", height = 3)

    return

In [25]:
alvo = ['life_expectancy']

In [26]:
# atributos (variáveis independentes)
atributos = ['population', 'hepatitisB', 'gdp', 'total_expenditure', 'alcohol', 'school'] 

In [29]:
dsa_get_pairs(novo_df, alvo, atributos, 3)

In [32]:
novo_df.count()

country              183
life_expectancy      183
year                 183
status               183
adult_mortality      183
inf_death            183
alcohol              182
hepatitisB            85
measles              183
bmi                  181
polio                180
diphtheria           180
hiv                  183
gdp                  154
total_expenditure    179
thinness_till19      181
thinness_till9       181
school               173
population           143
dtype: int64

In [30]:
valores_ausentes = novo_df.isnull().sum().sort_values(ascending = False)

In [31]:
valores_ausentes

hepatitisB           98
population           40
gdp                  29
school               10
total_expenditure     4
polio                 3
diphtheria            3
thinness_till9        2
thinness_till19       2
bmi                   2
alcohol               1
life_expectancy       0
measles               0
hiv                   0
inf_death             0
adult_mortality       0
status                0
year                  0
country               0
dtype: int64